In [1]:
import pandas as pd
from scholarly import scholarly, ProxyGenerator
from habanero import Crossref
from pyscopus import Scopus

scopus = Scopus("f130ed2ec5730526ea6c672a96600fd7")

# [Worked] Get Metadata based on Titles from Scopus

In [17]:
titles = pd.read_csv("snowballing_additional_backward.csv", sep=";")
titles.head()

,Title
0,Knowledge-based lifecycle management approach ...
1,Unified IoT ontology to enable interoperabilit...
2,Agile Manifesto
3,Development of a knowledge-based design suppor...
4,Framework for the agile development of innovat...


In [ ]:
ids = []
failed_titles = []
for title in titles["Title"]:
    try:
        result = scopus.search("Title(" + title + ")")
        ids.append(result["scopus_id"])
        print("success")
    except Exception as e:
        print("failed: ", e)
        failed_titles.append(title)

### persist scopus ids

In [20]:
id_test = []
for row in ids:
    if not row.empty:
        id_test.append(row[0])
id_list = pd.Series(id_test)
id_list.to_csv("scopus_ids_additional_backward.csv", sep=";", index=False)
failed_list = pd.Series(failed_titles)
failed_list.to_csv("scopus_failed_backward.csv", sep=";", index=False)

### load scopus ids from previous run, and get metadata for those ids

In [2]:
scopus_ids = pd.read_csv("scopus_ids_additional_backward.csv", sep=";", encoding="latin-1")

In [ ]:
columns = ["title", "abstract", "dc:publisher", "prism:aggregationType", "prism:publicationName", "prism:coverDate", "prism:url"]
result = pd.DataFrame(columns=columns)
failed_ids = []
for index, scopus_id in scopus_ids.iterrows():
    try:
        pub_info = scopus.retrieve_abstract(scopus_id[0])
        new_line = {}
        print("success")
        for column in columns:
            new_line[column] = pub_info[column]
        result = result.append(new_line, ignore_index=True)
    except Exception as ex:
        failed_ids.append(scopus_id)
        print("failed", ex)
        continue
    

In [38]:
pub_info = scopus.retrieve_abstract("85143907362")
new_line = {}
for column in columns:
            new_line[column] = pub_info[column]
result = result.append(new_line, ignore_index=True)

C:\Users\Daniel Lehner\AppData\Local\Temp\ipykernel_14724\3248324592.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_line, ignore_index=True)


## persist metadata in file finally.csv

In [23]:
result.to_csv("snowballing_metadata.csv", sep=";")

In [5]:
len(failed_ids)
pd.Series(failed_ids).to_csv("failed_ids.csv", sep=";", index=False)